In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
 
# Image dimensions
img_width, img_height = 150, 150 
 


In [5]:
def create_model(p, input_shape=(32, 32, 3)):
    # Initialising the CNN
    model = Sequential()
    # Convolution + Pooling Layer 
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Convolution + Pooling Layer 
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Convolution  followed by Pooling Layer 
    # increasing the number of filters as we have pooled out features reduced features. we can afford to apply 64 filters.
    
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Convolution + Pooling Layer 
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Flattening
    model.add(Flatten())
    # Fully connection ANN
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(p))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(p/2))
    model.add(Dense(1, activation='sigmoid'))
        # Compiling the CNN
    optimizer = Adam(lr=1e-3)
    metrics=['accuracy']
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=metrics)
    return model

In [6]:
def run_training(bs=32, epochs=10):
    
    train_datagen = ImageDataGenerator(rescale = 1./255, 
                                       shear_range = 0.2, 
                                       zoom_range = 0.2, 
                                       horizontal_flip = True)
    test_datagen = ImageDataGenerator(rescale = 1./255)
 
    training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (img_width, img_height),
                                                 batch_size = bs,
                                                 class_mode = 'binary')
                                                 
    test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (img_width, img_height),
                                            batch_size = bs,
                                            class_mode = 'binary')
                                            
    model = create_model(p=0.6, input_shape=(img_width, img_height, 3))                                  
    model.fit_generator(training_set,
                         steps_per_epoch=8000/bs,
                         epochs = epochs,
                         validation_data = test_set,
                         validation_steps = 2000/bs)
    return model


In [7]:
classifier = run_training(bs=32, epochs=100)#100 epochs

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/100
250/250 [==============================] - 340s 1s/step - loss: 0.6896 - acc: 0.5255 - val_loss: 0.6897 - val_acc: 0.5770
Epoch 2/100
250/250 [==============================] - 181s 726ms/step - loss: 0.6935 - acc: 0.5081 - val_loss: 0.6930 - val_acc: 0.5005
Epoch 3/100
250/250 [==============================] - 363s 1s/step - loss: 0.6932 - acc: 0.5052 - val_loss: 0.6891 - val_acc: 0.5090
Epoch 4/100
250/250 [==============================] - 190s 759ms/step - loss: 0.6889 - acc: 0.5275 - val_loss: 0.6895 - val_acc: 0.5000
Epoch 5/100
250/250 [==============================] - 287s 1s/step - loss: 0.6879 - acc: 0.5553 - val_loss: 0.6791 - val_acc: 0.5905
Epoch 6/100
250/250 [==============================] - 187s 749ms/step - loss: 0.6804 - acc: 0.5707 - val_loss: 0.6676 - val_acc: 0.6170
Epoch 7/100
250/250 [==============================] - 182s 729ms/step - loss: 0.6591 - acc: 0.6120 - v

250/250 [==============================] - 187s 749ms/step - loss: 0.2523 - acc: 0.8951 - val_loss: 0.3389 - val_acc: 0.8530
Epoch 61/100
250/250 [==============================] - 188s 752ms/step - loss: 0.2590 - acc: 0.8899 - val_loss: 0.3372 - val_acc: 0.8565
Epoch 62/100
250/250 [==============================] - 189s 754ms/step - loss: 0.2389 - acc: 0.8979 - val_loss: 0.3432 - val_acc: 0.8535
Epoch 63/100
250/250 [==============================] - 184s 738ms/step - loss: 0.2473 - acc: 0.8984 - val_loss: 0.3307 - val_acc: 0.8510
Epoch 64/100
250/250 [==============================] - 184s 737ms/step - loss: 0.2402 - acc: 0.9001 - val_loss: 0.3030 - val_acc: 0.8665
Epoch 65/100
250/250 [==============================] - 186s 745ms/step - loss: 0.2448 - acc: 0.9002 - val_loss: 0.3361 - val_acc: 0.8615
Epoch 66/100
250/250 [==============================] - 188s 750ms/step - loss: 0.2457 - acc: 0.8980 - val_loss: 0.3162 - val_acc: 0.8590
Epoch 67/100
250/250 [=========================

In [9]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (150, 150))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

NameError: name 'training_set' is not defined

In [10]:
import pickle
# save the model to disk
filename = 'cnn.sav'
pickle.dump(classifier, open(filename, 'wb'))

TypeError: can't pickle _thread.lock objects

In [11]:
# serialize model to JSON
model_json = classifier.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
classifier.save_weights("model.h5")
print("Saved model to disk")
 
# later...
"""
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")"""

Saved model to disk


'\n# load json and create model\njson_file = open(\'model.json\', \'r\')\nloaded_model_json = json_file.read()\njson_file.close()\nloaded_model = model_from_json(loaded_model_json)\n# load weights into new model\nloaded_model.load_weights("model.h5")\nprint("Loaded model from disk")'